# Technical Factor - Commodity Channel Index (CCI)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (high + low + close) / 3 AS TP,
        m_avg(TP, 12)            AS MA,
        m_avg(ABS(TP - MA), 12)  AS MD,
        (TP - MA) / (0.015 * MD) AS CCI,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(CCI > 0 AND m_lag(CCI, 1) <= 0, 1, 0) AS TRBY1,
        IF(CCI < 0 AND m_lag(CCI, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(CCI > 0 AND (CCI - m_lag(CCI, 1)) > 0, 1, 0) AS MTBY1,
        IF(CCI < 0 AND (CCI - m_lag(CCI, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(CCI < -100 AND (CCI - m_lag(CCI, 1)) > 0, 1, 0) AS RVBY1,
        IF(CCI > 100 AND (CCI - m_lag(CCI, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(CCI, 1) <= 100 AND CCI > 100, 1, 0) AS BKBY1,
        IF(m_lag(CCI, 1) >= -100 AND CCI < -100, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        TP,
        MA,
        MD,
        CCI,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,TP,MA,MD,CCI,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-11,600519.SH,11593.965906,11732.894411,138.340189,-66.950178,0,0,0,0,0,0,0,0
1,2025-02-12,600519.SH,11689.055836,11719.940151,139.412357,-14.768808,0,0,0,0,0,0,0,0
2,2025-02-13,600519.SH,11878.964164,11708.678429,139.690864,81.267894,1,0,1,0,0,0,0,0
3,2025-02-14,600519.SH,11975.085909,11707.562892,149.796507,119.060639,0,0,1,0,0,0,1,0
4,2025-02-17,600519.SH,12038.841236,11727.192274,172.306562,120.579258,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2025-12-25,600519.SH,11924.421463,11833.071533,93.865200,64.880225,0,0,1,0,0,0,0,0
217,2025-12-26,600519.SH,11947.622535,11862.313374,75.541405,75.286889,0,0,1,0,0,0,0,0
218,2025-12-29,600519.SH,11874.021078,11876.943169,61.168858,-3.184726,0,1,0,1,0,0,0,0
219,2025-12-30,600519.SH,11762.774194,11874.529770,65.311035,-114.075236,0,0,0,1,0,0,0,1
